[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremylongshore/claude-code-plugins-plus-skills/blob/main/tutorials/plugins/03-build-your-first-plugin.ipynb)

# Build Your First Plugin

**Learning Path**: Skills → **Plugins** → Orchestration  
**Level**: Intermediate  
**Time**: 45 minutes  
**Prerequisites**: [01-what-is-plugin](01-what-is-plugin.ipynb), [02-plugin-structure](02-plugin-structure.ipynb)

---

## What You'll Build

A complete **code-review-toolkit** plugin with:
- 2 skills (security-checker, style-analyzer)
- 1 slash command (/review)
- Enterprise-compliant structure (6767-c)
- Full documentation
- Validation passing

### Final Structure
```
code-review-toolkit/
├── .claude-plugin/
│   └── plugin.json
├── skills/
│   ├── security-checker/
│   │   └── SKILL.md
│   └── style-analyzer/
│       └── SKILL.md
├── commands/
│   └── review.md
├── README.md
└── LICENSE
```

---

## Step 1: Planning Your Plugin

Before writing code, answer these questions:

In [ ]:
# Plugin planning worksheet
plugin_plan = {
    "name": "code-review-toolkit",
    "purpose": "Automated code review with security and style checks",
    "target_users": "Developers who want consistent code reviews",
    "skills_needed": [
        "security-checker: Scan for OWASP Top 10 vulnerabilities",
        "style-analyzer: Check PEP 8, ESLint, etc."
    ],
    "commands_needed": [
        "/review: Quick code review of current file"
    ],
    "dependencies": "None (instruction plugin)",
    "category": "security"
}

print("PLUGIN PLANNING WORKSHEET")
print("=" * 60)
for key, value in plugin_plan.items():
    print(f"\n{key.upper().replace('_', ' ')}:")
    if isinstance(value, list):
        for item in value:
            print(f"  - {item}")
    else:
        print(f"  {value}")

print("\n✅ Plan complete! Ready to build.")

## Step 2: Create Directory Structure

First, create the plugin directory and all required subdirectories:

In [ ]:
from pathlib import Path
import json

def create_plugin_structure(plugin_name, output_dir="./"):
    """
    Create complete plugin directory structure.
    """
    base = Path(output_dir) / plugin_name
    
    # Create directories
    directories = [
        base / ".claude-plugin",
        base / "skills" / "security-checker",
        base / "skills" / "style-analyzer",
        base / "commands"
    ]
    
    for dir_path in directories:
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"✅ Created: {dir_path.relative_to(Path(output_dir))}")
    
    return base

# Create the structure (simulation)
print("CREATING PLUGIN STRUCTURE")
print("=" * 60)
plugin_root = create_plugin_structure("code-review-toolkit", "/tmp")
print(f"\n✅ Plugin structure created at: {plugin_root}")

## Step 3: Write plugin.json

The plugin manifest with all required fields:

In [ ]:
def generate_plugin_json(name, description, author_name, author_email, 
                        category="productivity", keywords=None):
    """
    Generate enterprise-compliant plugin.json.
    """
    if keywords is None:
        keywords = ["code-review", "quality"]
    
    plugin_json = {
        "name": name,
        "version": "1.0.0",
        "description": description,
        "author": {
            "name": author_name,
            "email": author_email
        },
        "license": "MIT",
        "keywords": keywords,
        "category": category,
        "repository": f"https://github.com/{author_name.lower().replace(' ', '-')}/{name}"
    }
    
    return plugin_json

# Generate for our plugin
plugin_json = generate_plugin_json(
    name="code-review-toolkit",
    description="Automated code review toolkit with security scanning and style analysis",
    author_name="Your Name",
    author_email="you@example.com",
    category="security",
    keywords=["code-review", "security", "style", "quality", "OWASP"]
)

print("PLUGIN.JSON")
print("=" * 60)
print(json.dumps(plugin_json, indent=2))

# Save to file
output_file = plugin_root / ".claude-plugin" / "plugin.json"
with open(output_file, 'w') as f:
    json.dump(plugin_json, f, indent=2)
print(f"\n✅ Saved to: {output_file}")

## Step 4: Create Skills

### Skill 1: security-checker

In [ ]:
security_checker_skill = """---
name: security-checker
description: |
  Scan code for security vulnerabilities based on OWASP Top 10.
  Use when: Reviewing code for security issues, pre-commit checks, security audits.
allowed-tools: Read, Grep, Bash(git:*)
version: 1.0.0
license: MIT
author: Your Name <you@example.com>
tags: [security, OWASP, vulnerability-scanning]
---

# Security Checker

Analyze code for common security vulnerabilities.

## OWASP Top 10 Checks

### 1. Injection Attacks
- SQL Injection: Look for string concatenation in queries
- Command Injection: Check for unsanitized user input in system calls
- LDAP Injection: Validate LDAP filter construction

**Pattern to detect**:
```python
# ❌ Vulnerable
query = f"SELECT * FROM users WHERE id = {user_id}"

# ✅ Safe  
query = "SELECT * FROM users WHERE id = ?"
```

### 2. Broken Authentication
- Weak password requirements
- Session fixation vulnerabilities
- Missing password hashing

### 3. Sensitive Data Exposure
- Hardcoded secrets/API keys
- Unencrypted sensitive data
- Logging sensitive information

**Check for**:
```python
# ❌ Hardcoded secrets
API_KEY = "sk_live_abc123..."

# ✅ Environment variables
API_KEY = os.getenv("API_KEY")
```

### 4. XML External Entities (XXE)
- Unsafe XML parsing
- External entity processing enabled

### 5. Broken Access Control
- Missing authorization checks
- Insecure direct object references

## Execution Steps

1. **Read target files** using Read tool
2. **Scan for patterns** listed above
3. **Check git history** for leaked secrets: `Bash(git:log -p)`
4. **Generate report**:
   - Critical: Immediate fix required
   - High: Fix before release
   - Medium: Address in next sprint
   - Low: Technical debt

## Output Format

```
Security Scan Results
=====================

🔴 CRITICAL (2):
  - file.py:42 - SQL Injection vulnerability
  - config.py:12 - Hardcoded API key

🟠 HIGH (1):
  - auth.py:88 - Weak password requirements

✅ No medium or low issues found.
```
"""

# Save skill
skill_file = plugin_root / "skills" / "security-checker" / "SKILL.md"
with open(skill_file, 'w') as f:
    f.write(security_checker_skill)

print("SECURITY-CHECKER SKILL")
print("=" * 60)
print(security_checker_skill[:500] + "...")
print(f"\n✅ Saved to: {skill_file}")

### Skill 2: style-analyzer

In [ ]:
style_analyzer_skill = """---
name: style-analyzer
description: |
  Analyze code style and consistency against best practices.
  Use when: Code reviews, pre-commit checks, maintaining code quality.
allowed-tools: Read, Grep
version: 1.0.0
license: MIT
author: Your Name <you@example.com>
tags: [style, linting, code-quality, best-practices]
---

# Style Analyzer

Check code against language-specific style guides.

## Supported Languages

### Python (PEP 8)
- Line length: ≤ 79 characters
- Indentation: 4 spaces (not tabs)
- Import order: stdlib → third-party → local
- Naming: snake_case for functions/variables, PascalCase for classes

### JavaScript (ESLint/Airbnb)
- Semicolons: required
- Quotes: single quotes for strings
- Indentation: 2 spaces
- Arrow functions: prefer over function expressions

### TypeScript (TSLint)
- Type annotations: required for public APIs
- Interface naming: no "I" prefix
- Enums: PascalCase

## Checks Performed

1. **Line Length**
   - Read file and check each line
   - Flag lines exceeding language limits

2. **Indentation**
   - Detect tabs vs spaces
   - Check consistency throughout file

3. **Naming Conventions**
   - Extract function/class names using Grep
   - Validate against language conventions

4. **Import Organization**
   - Check import order
   - Flag unused imports

5. **Documentation**
   - Public functions have docstrings/JSDoc
   - Classes have description comments

## Execution

1. Detect language from file extension
2. Load appropriate style rules
3. Read and analyze file
4. Generate styled report

## Output Format

```
Style Analysis: main.py
=======================

📏 Line Length (3 issues):
  - Line 42: 85 chars (limit: 79)
  - Line 108: 92 chars (limit: 79)

📝 Naming (1 issue):
  - Line 15: function 'ProcessData' should be 'process_data'

📚 Documentation (2 issues):
  - Line 20: Missing docstring for public function
  - Line 55: Class missing description

Score: 7/10 (Good)
```
"""

# Save skill
skill_file = plugin_root / "skills" / "style-analyzer" / "SKILL.md"
with open(skill_file, 'w') as f:
    f.write(style_analyzer_skill)

print("STYLE-ANALYZER SKILL")
print("=" * 60)
print(style_analyzer_skill[:500] + "...")
print(f"\n✅ Saved to: {skill_file}")

## Step 5: Create Slash Command

The `/review` command combines both skills:

In [ ]:
review_command = """---
description: Run comprehensive code review (security + style)
allowed-tools: Read, Grep, Bash(git:*)
---

Perform a comprehensive code review of the current file or directory.

## Steps

1. **Security Scan**
   - Use the `security-checker` skill
   - Check for OWASP Top 10 vulnerabilities
   - Scan git history for leaked secrets

2. **Style Analysis**
   - Use the `style-analyzer` skill
   - Check against language-specific style guides
   - Verify naming conventions and documentation

3. **Generate Report**
   - Combine results from both scans
   - Prioritize issues by severity
   - Provide actionable recommendations

## Output

```
Code Review Report
==================

File: main.py
Date: 2025-12-21

🔒 Security (2 issues):
  🔴 CRITICAL: Hardcoded API key (line 12)
  🟠 HIGH: SQL injection risk (line 42)

✨ Style (3 issues):
  - Line length violations (3)
  - Missing docstrings (2)

Overall Score: 6/10
Recommendation: Fix critical security issues before merging
```

Ask the user: "Which file or directory should I review?"
"""

# Save command
command_file = plugin_root / "commands" / "review.md"
with open(command_file, 'w') as f:
    f.write(review_command)

print("/REVIEW COMMAND")
print("=" * 60)
print(review_command)
print(f"\n✅ Saved to: {command_file}")

## Step 6: Write Documentation

In [ ]:
readme_content = """# Code Review Toolkit

Automated code review plugin for Claude Code with security scanning and style analysis.

## Features

- 🔒 **Security Checker**: OWASP Top 10 vulnerability scanning
- ✨ **Style Analyzer**: Language-specific style guide enforcement
- ⚡ **Quick Command**: `/review` for instant code reviews

## Installation

```bash
# Manual installation
cp -r code-review-toolkit ~/.claude/plugins/

# Or via git
cd ~/.claude/plugins
git clone https://github.com/your-name/code-review-toolkit.git
```

## Usage

### Quick Review (Command)

```
/review
```

Claude will ask which file to review and run both security and style checks.

### Security Check Only (Skill)

```
Scan this file for security vulnerabilities
```

Claude automatically uses the `security-checker` skill.

### Style Check Only (Skill)

```
Check code style in main.py
```

Claude automatically uses the `style-analyzer` skill.

## Supported Languages

- Python (PEP 8)
- JavaScript (ESLint/Airbnb)
- TypeScript (TSLint)
- More coming soon!

## Skills Included

### security-checker
- SQL injection detection
- Command injection scanning
- Hardcoded secrets detection
- Authentication issue detection
- Access control validation

### style-analyzer
- Line length checking
- Indentation validation
- Naming convention enforcement
- Import organization
- Documentation completeness

## License

MIT License - see LICENSE file for details

## Author

Your Name <you@example.com>

## Contributing

Contributions welcome! Please open an issue or PR.
"""

# Save README
readme_file = plugin_root / "README.md"
with open(readme_file, 'w') as f:
    f.write(readme_content)

# Create LICENSE
license_content = """MIT License

Copyright (c) 2025 Your Name

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

license_file = plugin_root / "LICENSE"
with open(license_file, 'w') as f:
    f.write(license_content)

print("DOCUMENTATION")
print("=" * 60)
print(f"✅ Created README.md ({len(readme_content)} chars)")
print(f"✅ Created LICENSE (MIT)")

## Step 7: Validate Your Plugin

In [ ]:
import os

def validate_plugin(plugin_path):
    """
    Complete plugin validation.
    """
    errors = []
    warnings = []
    success = []
    
    plugin_root = Path(plugin_path)
    
    # Check required files
    required_files = [
        ".claude-plugin/plugin.json",
        "README.md",
        "LICENSE"
    ]
    
    for file_path in required_files:
        full_path = plugin_root / file_path
        if full_path.exists():
            success.append(f"✅ {file_path} present")
        else:
            errors.append(f"❌ Missing: {file_path}")
    
    # Validate plugin.json
    plugin_json_path = plugin_root / ".claude-plugin" / "plugin.json"
    if plugin_json_path.exists():
        with open(plugin_json_path) as f:
            try:
                plugin_data = json.load(f)
                
                # Check required fields
                required = ["name", "version", "description", "author", "license", "keywords"]
                for field in required:
                    if field in plugin_data:
                        success.append(f"✅ plugin.json has '{field}'")
                    else:
                        errors.append(f"❌ Missing field in plugin.json: {field}")
                
            except json.JSONDecodeError as e:
                errors.append(f"❌ Invalid JSON in plugin.json: {e}")
    
    # Check skills
    skills_dir = plugin_root / "skills"
    if skills_dir.exists():
        skill_dirs = [d for d in skills_dir.iterdir() if d.is_dir()]
        for skill_dir in skill_dirs:
            skill_md = skill_dir / "SKILL.md"
            if skill_md.exists():
                success.append(f"✅ Skill: {skill_dir.name}")
            else:
                errors.append(f"❌ Missing SKILL.md in {skill_dir.name}/")
    
    # Check commands
    commands_dir = plugin_root / "commands"
    if commands_dir.exists():
        command_files = list(commands_dir.glob("*.md"))
        for cmd_file in command_files:
            success.append(f"✅ Command: /{cmd_file.stem}")
    
    # Print results
    print("PLUGIN VALIDATION RESULTS")
    print("=" * 60)
    
    if success:
        print("\nPassed Checks:")
        for item in success:
            print(f"  {item}")
    
    if warnings:
        print("\nWarnings:")
        for item in warnings:
            print(f"  ⚠️ {item}")
    
    if errors:
        print("\nErrors:")
        for item in errors:
            print(f"  {item}")
        print("\n❌ VALIDATION FAILED")
    else:
        print("\n✅ ALL CHECKS PASSED!")
        print("\nYour plugin is ready to use!")
    
    return len(errors) == 0

# Validate our plugin
validate_plugin(plugin_root)

## Step 8: Test Your Plugin

### Installation Test

```bash
# Copy to Claude plugins directory
cp -r /tmp/code-review-toolkit ~/.claude/plugins/

# Restart Claude Code to load the plugin
```

### Functionality Test

1. **Test slash command**:
   ```
   /review
   ```
   
2. **Test security skill**:
   ```
   Scan main.py for security vulnerabilities
   ```
   
3. **Test style skill**:
   ```
   Check code style in app.js
   ```

### Expected Behavior

- Claude automatically discovers and loads your plugin
- `/review` command appears in command palette
- Skills activate based on user intent matching descriptions
- All tools work within specified `allowed-tools`

## Key Takeaways

### What You Built

1. ✅ **Complete plugin** - All required components present
2. ✅ **2 skills** - security-checker + style-analyzer
3. ✅ **1 command** - /review for quick access
4. ✅ **Documentation** - README + LICENSE
5. ✅ **Validation** - All enterprise standards met

### Enterprise Checklist

- [x] kebab-case name
- [x] SemVer version
- [x] Complete plugin.json
- [x] Skills with YAML frontmatter
- [x] allowed-tools specified
- [x] README documentation
- [x] LICENSE file
- [x] Proper directory structure
- [x] No hardcoded secrets
- [x] Portable paths

### Skills Learned

- Plugin planning and design
- Directory structure creation
- plugin.json authoring
- Skill writing for specific domains
- Slash command creation
- Documentation best practices
- Validation and testing

---

## Next Steps

1. **Enhance skills** - Add more security checks or style rules
2. **Add agents** - Create specialized review agents
3. **MCP integration** - [04-mcp-server-plugins.ipynb](04-mcp-server-plugins.ipynb)
4. **Publish** - Share on marketplace or GitHub
5. **Iterate** - Gather feedback and improve

---

*Enterprise Standards Compliant • 6767-c • Version 1.0.0 • MIT License*